# Use case :


#### Experiments

In [1]:
from evaluate import (GeneralExperiment,
                      SpecializedExperiment)

exp_Eg = GeneralExperiment(preds=None, target=None)
exp_Es = SpecializedExperiment(preds=None, target=None)

#### Models

In [2]:
from models import (EpiLLM,
                    BaseModel,
                    Baseline)


for modclass in [EpiLLM, BaseModel, Baseline]:
    ## instantiate
    model = modclass()

    ## load params/ pipeline
    model.load_model()

    ## train
    model.train()

    ## test
    pred = model.predict()


#### EpiScape

In [3]:
from episcape import (PatientGenerator,
                      SDeHRGenerator,
                      PipelineGenerator)